Imports

In [65]:
!pip install ml-collections

In [67]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import time
import torch
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
import ml_collections
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from imblearn.metrics import specificity_score


print('Is Colab using GPU?', torch.cuda.is_available())
# Turn on GPU, by clicking Runtime -> change runtime type -> T4 GPU

Is Colab using GPU? False


In [68]:
############################# SETTINGS ##########################

config = ml_collections.ConfigDict()

# Device
config.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', config.device)

# Hyperparameters
config.random_seed = 123
config.learning_rate = 0.001
config.num_epochs = 1000

# Architecture
config.num_features = 1215
config.latent_dim = 32

Device: cpu


In [69]:
!gdown --fuzzy https://drive.google.com/file/d/1CQgnLkSVKEPpm9NPLW03scyQy8aRp0qe/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1mBD0zUCuYHtWsRcp6wkWiP76qpCeQkYY/view?usp=sharing!gdown
!gdown --fuzzy https://drive.google.com/file/d/17D7MuSJpVjYBafeeFYaDw-a6Mk5Laa7B/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1O_2vgPYdYzJrobKahyiReaepFi1kbhvY/view?usp=sharing
!gdown --fuzzy https://drive.google.com/file/d/1uq3_eOzI7xN5mYE8EEsC4-jnrIZfBEig/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1CQgnLkSVKEPpm9NPLW03scyQy8aRp0qe
To: /content/2022-02-18-TeraTox-commercial-logFC.gct
100% 4.88M/4.88M [00:00<00:00, 23.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mBD0zUCuYHtWsRcp6wkWiP76qpCeQkYY
To: /content/2022-02-18-TeraTox-commercial-pScore.gct
100% 4.70M/4.70M [00:00<00:00, 24.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=17D7MuSJpVjYBafeeFYaDw-a6Mk5Laa7B
To: /content/2022-02-18-TeraTox-commercial-featureData.txt
100% 72.8k/72.8k [00:00<00:00, 3.88MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O_2vgPYdYzJrobKahyiReaepFi1kbhvY
To: /content/2022-02-18-TeraTox-commercial-phenoData.txt
100% 21.4k/21.4k [00:00<00:00, 47.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uq3_eOzI7xN5mYE8EEsC4-jnrIZfBEig
To: /content/2021-06-10-gcGeneFactorAnno-withPositiveCoefs.tsv
100% 6.98k/6.98k [00:00<00:00, 25.0MB/s]


***Data***

In [70]:
# phenoData['Teratogenicity'].to_numpy()

In [71]:
phenoData = pd.read_csv("./2022-02-18-TeraTox-commercial-phenoData.txt", sep='\t')
all_label = (phenoData['Teratogenicity'].to_numpy() == 'Positive').astype(float) # 0 is negative, 1 is positive
logFCdata = pd.read_csv("./2022-02-18-TeraTox-commercial-logFC.gct", sep='\t',skiprows=2)
logFCdata = logFCdata.drop(columns=['Description', 'NAME'])

all_data = logFCdata.to_numpy().T #[num data, data dim]


all_indices = np.arange(all_data.shape[0])


In [72]:
np.random.shuffle(all_indices)

train_indices = all_indices[: int(len(all_indices) * 0.8)]
val_indices = all_indices[int(len(all_indices) * 0.8): ]

train_data = torch.Tensor(all_data[train_indices, :])
train_label = all_label[train_indices]


val_data = torch.Tensor(all_data[val_indices, :])
val_label = all_label[val_indices]



Model

In [73]:
############################# MODEL ##########################

class Autoencoder(torch.nn.Module):
    def __init__(self, config):
        super(Autoencoder, self).__init__()

        ### ENCODER

        self.linear_1 = torch.nn.Linear(config.num_features, config.latent_dim)

        ### DECODER
        self.linear_2 = torch.nn.Linear(config.latent_dim, config.num_features)
        self.linear_1.weight.detach().normal_(0.0, 0.1)
        self.linear_1.bias.detach().zero_()

    def encoder(self, x):
        encoded = self.linear_1(x)
        encoded = F.leaky_relu(encoded)
        return encoded

    def decoder(self, encoded_x):
        logits = self.linear_2(encoded_x)
        # decoded = torch.sigmoid(logits)
        decoded = logits
        return decoded


    def forward(self, x):

        ### ENCODER
        encoded = self.encoder(x)

        ### DECODER
        decoded = self.decoder(encoded)

        return decoded


torch.manual_seed(config.random_seed)
model = Autoencoder(config)
model = model.to(config.device)

optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

In [74]:
latent = model.encoder(train_data)
recon = model.decoder(latent)

def compute_loss(recon, input):
  squared_error = (recon - input) ** 2
  mse = squared_error.mean()
  return mse

# ((model(train_data_plain) - train_data_plain ) ** 2).mean()

# compute_loss(model(train_data_plain), train_data_plain)

Training

In [ ]:
start_time = time.time()
for epoch in range(config.num_epochs):

    ### FORWARD AND BACK PROP
    recon = model(train_data)
    loss = compute_loss(recon, train_data)
    optimizer.zero_grad()

    loss.backward()

    ### UPDATE MODEL PARAMETERS
    optimizer.step()

    with torch.no_grad():
        recon = model(val_data)
        val_loss = compute_loss(recon, val_data)

    ### LOGGING
    print('Epoch: %03d/%03d | Train loss: %.4f | Val loss: %.4f'
            %(epoch+1, config.num_epochs, loss.item(), val_loss.item()))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

In [76]:
with torch.no_grad():
  train_latent = model.encoder(train_data)
  val_latent = model.encoder(val_data)

# Classify on original gene expressions

In [79]:
rf_raw = RandomForestClassifier(n_estimators=100, max_depth=5, n_jobs=-1).fit(np.array(train_data), train_label)

print(f'Train Accuracy: {rf_raw.score(np.array(train_data), train_label)*100}%')
print(f'Val Accuracy: {rf_raw.score(np.array(val_data), val_label)*100}%')

Train Accuracy: 100.0%
Val Accuracy: 72.72727272727273%


In [80]:
y_pred_raw = rf.predict(np.array(val_data))
f1_raw = f1_score(val_label, y_pred_raw )
precision_raw = precision_score(val_label, y_pred_raw )
recall_raw = recall_score(val_label, y_pred_raw )
specificity_raw = specificity_score(val_label, y_pred_raw )

0.7924528301886793 0.7241379310344828 0.875 0.6


In [85]:
print (f'f1_raw: {f1_raw * 100}%')
print (f'precision_raw: {precision_raw * 100}%')
print (f'recall_raw: {recall_raw * 100}%%')
print (f'specificity_raw: {specificity_raw * 100}%')

f1_raw: 79.24528301886792%
precision_raw: 72.41379310344827%
recall_raw: 87.5%%
specificity_raw: 60.0%


# Classify on learned latents

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=4, n_jobs=-1).fit(np.array(train_latent), train_label)
print(f'Train Accuracy: {rf.score(np.array(train_latent), train_label)*100}%')
print(f'Val Accuracy: {rf.score(np.array(val_latent), val_label)*100}%')

In [ ]:
y_pred = rf.predict(np.array(val_latent))

In [ ]:
f1_score(val_label, y_pred )

In [ ]:
precision_score(val_label, y_pred )

In [ ]:
recall_score(val_label, y_pred )

In [ ]:
specificity_score(val_label, y_pred )